In [21]:
import os
import pickle
import json
import io
import copy
import hashlib
import time

import tqdm

In [4]:
json_config_file = 'webconfig.json'

    # sample json content

# {
#     "game_obj_list_filepath": "tmp_pickle_files/game_obj_list-1650280733912302100.pickle",
#     "game_binary_url_list_filepath": "tmp_pickle_files/game_binary_url_list-1650280741999342000.pickle",
#     "image_url_list_filepath": "tmp_pickle_files/image_url_list-1650280741968099500.pickle"
# }

content_bs = open(json_config_file, 'rb').read()
content_str = content_bs.decode('utf-8')

In [5]:
config_obj = json.loads(content_str)

In [6]:
game_obj_list_filepath = config_obj['game_obj_list_filepath']
with open(game_obj_list_filepath, 'rb') as infile:
    game_obj_list = pickle.load(infile)

game_binary_url_list_filepath = config_obj['game_binary_url_list_filepath']
with open(game_binary_url_list_filepath, 'rb') as infile:
    game_binary_url_list = pickle.load(infile)
image_url_list_filepath = config_obj['image_url_list_filepath']

with open(image_url_list_filepath, 'rb') as infile:
    image_url_list = pickle.load(infile)

# game_obj_list

In [7]:
number_of_game_obj = len(game_obj_list)
number_of_game_obj

11732

In [8]:
game_obj_list[0]

{'url': 'http://www.7723.cn/download/10819.htm',
 'name': '星际车手',
 'banner_image': {'url': 'http://image.7723.cn/wuza/picp/201215O5239542.jpg'},
 'gameplay_image_list': [{'url': 'http://image.7723.cn/wuza/picp/201215O5239542.jpg'},
  {'url': 'http://images.7723.cn/wuza/picp/201215O5239522.gif'},
  {'url': 'http://images.7723.cn/wuza/picp/201215O5242894.gif'}],
 'binary_info_list': [{'url': 'http://www.7723.cn/downb.asp?idd=13&id=10819&ksp=1',
   'description': '诺基亚 N73系列(240×320)\nN73 5320 5320XM 5320di_XM 5630XM 5700 5700XM 5710XM 5730XM 6110 6110N 6120 6120C 6120ci 6121 6122C 6124C 6210S 6210ci 6220C 6290 6650F 6700S 6702S 6710N 6720C 6730c 6788 6788I 6790 C5 C5-01 E101 E50 E51 E52 E55 E65 E66 E75 X5-00 X5-01 N71 N73ie N75 N76 N76-1 N77 N78 N79 N79 Eco N81 N81 8GB N82 N85 N86 N92 N93 N93I N95 N958G N95 8GB N95-3 NAM N96'},
  {'url': 'http://www.7723.cn/downb.asp?idd=13&id=10819&ksp=2',
   'description': '诺基亚 N7370系列(240×320)\n7370 2700C 2710C 2710N 2730C 3120C 3208C 3600S 3602S 3610A

In [14]:
def hash_url(url: str):
    hash_str = hashlib.md5(url.encode('utf-8')).hexdigest()
    hash_str = hash_str.lower()
    return hash_str

In [18]:
modified_game_obj_list = []

for game_obj_index in tqdm.tqdm(range(number_of_game_obj)):
    if os.path.exists('stop'):
        break

    game_obj = game_obj_list[game_obj_index]
    # deep copy the object
    game_obj = copy.deepcopy(game_obj)

    # banner image
    if 'banner_image' in game_obj:
        if 'url' in game_obj['banner_image']:
            image_url = game_obj['banner_image']['url']
            url_hash = hash_url(image_url)
            game_obj['banner_image']['url_hash'] = url_hash
            # TODO add content hash
    if 'gameplay_image_list' in game_obj:
        gameplay_image_list = game_obj['gameplay_image_list']
        for gameplay_image_index in range(len(gameplay_image_list)):
            gameplay_image = gameplay_image_list[gameplay_image_index]
            if 'url' in gameplay_image:
                image_url = gameplay_image['url']
                url_hash = hash_url(image_url)
                gameplay_image['url_hash'] = url_hash
                # TODO add content hash

    # binary
    if 'binary_info_list' in game_obj:
        binary_info_list = game_obj['binary_info_list']
        for binary_info_index in range(len(binary_info_list)):
            binary_info = binary_info_list[binary_info_index]
            if 'url' in binary_info:
                binary_url = binary_info['url']
                url_hash = hash_url(binary_url)
                binary_info['url_hash'] = url_hash
                # TODO add content hash

    modified_game_obj_list.append(game_obj)

100%|██████████████████████████████████████████████████████████████████████████| 11732/11732 [00:02<00:00, 4213.70it/s]


In [19]:
len(modified_game_obj_list)

11732

In [20]:
len(game_obj_list)

11732

In [22]:
modified_game_obj_list_log_filepath = f'modified_game_obj_list-{time.time_ns()}.pickle'
print(modified_game_obj_list_log_filepath)

with open(modified_game_obj_list_log_filepath, mode='wb') as outfile:
    pickle.dump(modified_game_obj_list, outfile)

modified_game_obj_list-1653552476412252800.pickle


# image_url_list

In [23]:
modified_image_url_list = []

for image_url in tqdm.tqdm(image_url_list):
    url_hash = hash_url(image_url)
    modified_image_url_list.append({
        'url': image_url,
        'url_hash': url_hash
    })

100%|████████████████████████████████████████████████████████████████████████| 40337/40337 [00:00<00:00, 282236.67it/s]


In [24]:
modified_image_url_list_log_filepath = f'modified_image_url_list-{time.time_ns()}.pickle'
print(modified_image_url_list_log_filepath)
with open(modified_image_url_list_log_filepath, mode='wb') as outfile:
    pickle.dump(modified_image_url_list, outfile)

modified_image_url_list-1653552776937727500.pickle


# game_binary_url_list

In [25]:
modified_game_binary_url_list = []

for binary_url in tqdm.tqdm(game_binary_url_list):
    url_hash = hash_url(binary_url)
    modified_game_binary_url_list.append({
        'url': binary_url,
        'url_hash': url_hash
    })

100%|██████████████████████████████████████████████████████████████████████| 153953/153953 [00:00<00:00, 300275.10it/s]


In [26]:
modified_game_binary_url_list_log_filepath = f'modified_game_binary_url_list-{time.time_ns()}.pickle'
print(modified_game_binary_url_list_log_filepath)
with open(modified_game_binary_url_list_log_filepath, mode='wb') as outfile:
    pickle.dump(modified_game_binary_url_list, outfile)

modified_game_binary_url_list-1653552857468686500.pickle
